<a href="https://colab.research.google.com/github/oebyakarah/PDF-QA-Agent/blob/main/PDF_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Question-Answering Agent
This is An AI tool that lets you upload a PDF and ask questions about it using Retrieval-Augmented Generation .


In [ ]:
!pip install langchain langchain_community
!pip install pypdf faiss-cpu
!pip install transformers sentence-transformers
!pip install huggingface_hub

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!ls /content

In [ ]:
!pip install requests==2.32.4 --force-reinstall

In [ ]:
!pip show langchain langchain_community pypdf faiss-cpu transformers sentence-transformers requests

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "/content/AI_Whitepaper_Portfolio.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"PDF loaded! Total pages: {len(documents)}")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(chunks, embeddings)

In [ ]:
!pip install -q langchain langchain_community transformers faiss-cpu pypdf

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

query = "Summarize the main points from chapter 1."
result = qa_chain.run(query)
print(result)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
query = "Summarize the Methodolgy section."
result = qa_chain.run(query)
print(result)

In [11]:
import textwrap

def wrap_output(text, width=100):
    return "\n".join(textwrap.wrap(str(text), width=width))

questions = [
    "Summarize the Methodology section.",
    "What are the key findings?",
    "Explain the case study in simple terms."
]

for question in questions:
    raw_answer = qa_chain.run(question)
    wrapped_answer = wrap_output(raw_answer)
    print("Question:", question)
    print("Answer:\n", wrapped_answer)
    print("\n" + "="*80 + "\n")

Question: Summarize the Methodology section.
Answer:
 Retrieval-Augmented Generation (RAG): A Practical Whitepaper Prepared by: [Your Name Here] Date:
September 2025 This paper provides an overview of Agentic AI systems and their integration with
Retrieval-Augmented Generation (RAG). It covers the core concepts, methodologies, and a step-by-step
case study. This whitepaper is designed to demonstrate skills in Natural Language Processing (NLP),
LangChain pipelines, and modern LLM techniques for real-world applications.


Question: What are the key findings?
Answer:
 The system successfully answered domain-specific queries with a 92% accuracy rate in a controlled
test environment. Compared to a baseline non-RAG LLM, our system showed a 40% improvement in factual
correctness and significantly reduced hallucinations. Future Work Future improvements will focus on:
- Scaling to multi-modal inputs (text, images, audio). - Implementing advanced reasoning through
ReAct and tool-chaining. - Depl